In [ ]:
from scripts.user_satisfaction_analysis import (
    calculate_satisfaction,
    predict_satisfaction,
    assign_engagement_score,
assign_experience_score,
export_to_mysql
)
from scripts.db import retrieve_data_from_database
from scripts.data_clustering import apply_kmeans_clustering

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [ ]:
# Load data from the database
df = retrieve_data_from_database()

In [ ]:
# Normalize features
engagement_features = [
    "session_frequency",
    "session_duration",
    "total_traffic",
]
experience_features = [
    "Avg RTT DL (ms)",
    "Avg RTT UL (ms)",
    "Avg Bearer TP DL (kbps)",
    "Avg Bearer TP UL (kbps)",
    "TCP DL Retrans. Vol (Bytes)",
    "TCP UL Retrans. Vol (Bytes)",
]

In [ ]:
scaler = MinMaxScaler()
df[engagement_features] = scaler.fit_transform(df[engagement_features])
df[experience_features] = scaler.fit_transform(df[experience_features])

In [ ]:
# Engagement Clustering
kmeans_engagement = KMeans(n_clusters=3, random_state=42)
df['engagement_cluster'] = kmeans_engagement.fit_predict(df[engagement_features])
engagement_centers = kmeans_engagement.cluster_centers_

In [ ]:
# Assign engagement score
df = assign_engagement_score(df, engagement_features, engagement_centers)

In [ ]:
# Experience Clustering
kmeans_experience = KMeans(n_clusters=3, random_state=42)
df['experience_cluster'] = kmeans_experience.fit_predict(df[experience_features])
experience_centers = kmeans_experience.cluster_centers_

In [ ]:
# Assign experience score
df = assign_experience_score(df, experience_features, experience_centers)

In [ ]:
# Calculate satisfaction
df = calculate_satisfaction(df)

In [ ]:
# Report top 10 satisfied customers
top_10_satisfied = df.nlargest(10, 'satisfaction_score')[['IMSI', 'satisfaction_score']]
print("Top 10 Satisfied Customers:")
print(top_10_satisfied)

In [ ]:
# Predict satisfaction
model = predict_satisfaction(df, engagement_features + experience_features)
print("Regression model coefficients:", model.coef_)

In [ ]:
# Cluster engagement and experience
df, kmeans_final = apply_kmeans_clustering(df, n_clusters=2, columns=['engagement_score', 'experience_score'])

In [ ]:
# Aggregate satisfaction and experience scores per cluster
cluster_aggregates = df.groupby('cluster')[['satisfaction_score', 'experience_score']].mean()
print("Cluster Aggregates:")
print(cluster_aggregates)

In [ ]:
# Export to MySQL
export_to_mysql(df[['IMSI', 'engagement_score', 'experience_score', 'satisfaction_score']], "telecom_db", "satisfaction_scores")
